In [109]:
#CLEANING

In [110]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

df = pd.read_csv('kcpd_cleaned_data.csv', encoding='utf-8', low_memory=False)
df = df[~df['Zip Code'].str.contains('clay', case=False)]
df = df[df['Area'] != 'OSPD']
df['Zip Code'] = df['Zip Code'].astype(float).round(0).astype(int).astype(str)
df.dropna(how='any', inplace=True)
df

,Report_No,Reported_Date,From_Date,From_Time,Offense,IBRS,Description,Beat,Address,Zip Code,Area,DVFlag,Involvement,Firearm Used Flag
0,KC21055624,2021-08-21,08/21/2021,19:45,Murder,09A,Murder,332,3500 AGNES AVE,64128,EPD,False,VIC,True
1,KC21012401,2021-02-24,02/24/2021,11:35,Stealing from Building/Residence,23D,Theft From Building,234,8800 STATE LINE RD,64114,MPD,False,SUS,False
2,KC21017210,2021-03-17,03/17/2021,15:35,Assault (Aggravated),13A,Aggravated Assault,631,NE PARVIN RD and I 29 HWY,64116,SCP,False,ARR CHA,True
3,KC21010791,2021-02-17,02/11/2021,20:30,Stolen Auto,240,Motor Vehicle Theft,211,4200 E 53RD ST,64130,MPD,False,SUS,False
4,KC21012025,2021-02-22,02/22/2021,17:55,Assault (Aggravated),13A,Aggravated Assault,222,E 52ND ST and EUCLID AVE,64130,MPD,False,VIC,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79424,KC21083342,2021-12-08,12/08/2021,05:00,Stealing from Auto (Theft from Auto),23F,Theft From Motor Vehicle,114,700 W 24TH ST,64108,CPD,False,VIC,False
79425,KC21087253,2021-12-25,12/24/2021,21:00,Domestic Violence Assault (Non-Aggravated),90Z,All Other Offenses,532,10900 BLUE RIDGE BLVD,64134,SPD,True,VIC,False
79426,KC21083962,2021-12-10,12/10/2021,20:15,Property Damage,290,Vandalism/Destruction of Property,643,9800 NE BARRY RD,64157,SCP,False,SUS,False
79427,KC21084337,2021-12-12,12/11/2021,02:30,Domestic Violence Assault (Aggravated),13A,Aggravated Assault,143,4300 CAMPBELL ST,64110,CPD,True,SUS,False


In [111]:
df[['DVFlag','Firearm Used Flag']].astype(int)
df.dtypes

Report_No            object
Reported_Date        object
From_Date            object
From_Time            object
Offense              object
IBRS                 object
Description          object
Beat                 object
Address              object
Zip Code             object
Area                 object
DVFlag                 bool
Involvement          object
Firearm Used Flag      bool
dtype: object

In [112]:
df.dropna

<bound method DataFrame.dropna of         Report_No Reported_Date   From_Date From_Time  \
0      KC21055624    2021-08-21  08/21/2021     19:45   
1      KC21012401    2021-02-24  02/24/2021     11:35   
2      KC21017210    2021-03-17  03/17/2021     15:35   
3      KC21010791    2021-02-17  02/11/2021     20:30   
4      KC21012025    2021-02-22  02/22/2021     17:55   
...           ...           ...         ...       ...   
79424  KC21083342    2021-12-08  12/08/2021     05:00   
79425  KC21087253    2021-12-25  12/24/2021     21:00   
79426  KC21083962    2021-12-10  12/10/2021     20:15   
79427  KC21084337    2021-12-12  12/11/2021     02:30   
79428  KC21076282    2021-11-09  11/09/2021     09:00   

                                          Offense IBRS  \
0                                          Murder  09A   
1                Stealing from Building/Residence  23D   
2                            Assault (Aggravated)  13A   
3                                     Stolen Auto

In [113]:
X = df[['Beat','Zip Code','Area', 'DVFlag','Firearm Used Flag']]
X

,Beat,Zip Code,Area,DVFlag,Firearm Used Flag
0,332,64128,EPD,False,True
1,234,64114,MPD,False,False
2,631,64116,SCP,False,True
3,211,64130,MPD,False,False
4,222,64130,MPD,False,False
...,...,...,...,...,...
79424,114,64108,CPD,False,False
79425,532,64134,SPD,True,False
79426,643,64157,SCP,False,False
79427,143,64110,CPD,True,False


In [114]:
#PREPROCESSING

In [115]:
ohe_crime = pd.get_dummies(df['Description'])
ohe_crime

,Aggravated Assault,All Other Larceny,All Other Offenses,Animal Cruelty,Arson,Assisting or Promoting Prostitution,Bad Checks,Burglary/Breaking and Entering,Commercial Sex Acts,Counterfeiting / Forgery,...,Stolen Property Offenses,Theft From Building,Theft From Coin-Operated Machine,Theft From Motor Vehicle,Theft of Vehicle Parts and Accessories,Trespass of Real Property,Vandalism/Destruction of Property,Weapon Law Violations,Welfare Fraud,Wire Fraud
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79424,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
79425,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
79427,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
ohe_area = pd.get_dummies(df['Area'])
ohe_area

,CPD,EPD,MPD,NPD,SCP,SPD
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,0,0,1,0
3,0,0,1,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
79424,1,0,0,0,0,0
79425,0,0,0,0,0,1
79426,0,0,0,0,1,0
79427,1,0,0,0,0,0


In [117]:
X = pd.concat([X[['Beat', 'Zip Code', 'DVFlag', 'Firearm Used Flag']], ohe_crime], axis=1)
X

,Beat,Zip Code,DVFlag,Firearm Used Flag,Aggravated Assault,All Other Larceny,All Other Offenses,Animal Cruelty,Arson,Assisting or Promoting Prostitution,...,Stolen Property Offenses,Theft From Building,Theft From Coin-Operated Machine,Theft From Motor Vehicle,Theft of Vehicle Parts and Accessories,Trespass of Real Property,Vandalism/Destruction of Property,Weapon Law Violations,Welfare Fraud,Wire Fraud
0,332,64128,False,True,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,234,64114,False,False,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,631,64116,False,True,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,211,64130,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,222,64130,False,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79424,114,64108,False,False,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
79425,532,64134,True,False,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79426,643,64157,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
79427,143,64110,True,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
# X = X[['Beat', 'DVFlag', 'Zip Code','Firearm Used Flag', 'CPD', 'EPD', 'MPD',
#        'NPD', 'SCP', 'SPD']]
# X

In [119]:
y = df['Description']
y.unique()

array(['Murder', 'Theft From Building', 'Aggravated Assault',
       'Motor Vehicle Theft', 'Simple Assault',
       'Counterfeiting / Forgery', 'Identity Theft', 'Robbery',
       'Shoplifting', 'Prostitution', 'Theft From Motor Vehicle',
       'Embezzlement', 'Disorderly Conduct', 'Forcible Sodomy',
       'All Other Larceny', 'Trespass of Real Property', 'Intimidation',
       'Vandalism/Destruction of Property', 'Family Offenses, Nonviolent',
       'Forcible Rape', 'Arson', 'Drug / Narcotic Violations',
       'Burglary/Breaking and Entering', 'Forcible Fondling',
       'All Other Offenses', 'Driving Under the Influence',
       'Stolen Property Offenses', 'Weapon Law Violations', 'Wire Fraud',
       'False Pretenses / Swindle', 'Drug Equipment Violations',
       'Theft of Vehicle Parts and Accessories', 'Statutory Rape',
       'Credit Card / Automatic Teller', 'Kidnapping / Abduction',
       'Bad Checks', 'Extortion / Blackmail', 'Impersonation',
       'Pornography / Obsce

In [120]:
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (79283, 60)
y shape: (79283,)


In [121]:
#Machine Learning

In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [123]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(estimators)
pipe
Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])



Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

In [128]:
y_test

57159                         Intimidation
37625                          Shoplifting
46493            Trespass of Real Property
68817                  Motor Vehicle Theft
1043     Vandalism/Destruction of Property
                       ...                
62939            Trespass of Real Property
63845                      Forcible Sodomy
64880    Vandalism/Destruction of Property
22845             Theft From Motor Vehicle
58609             Counterfeiting / Forgery
Name: Description, Length: 23785, dtype: object

In [124]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
clf.fit(X, y)
SGDClassifier(max_iter=5)

/Users/brettsquires/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDClassifier(max_iter=5)

In [125]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
)


y_pred = clf.predict(X_test)
accuray = accuracy_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test, average="weighted")

print("Accuracy:", accuray)
print("F1 Score:", f1)

Accuracy: 0.1396258145890267
F1 Score: 0.2450342118995117
